# Week 3 Assignment-Part 3

In [1]:
import folium
import pandas as pd
import numpy as np

## 1. Get the desired dataset

#### Load the geographical information of Toronto

In [4]:
file_path='geoinfo.csv'

In [5]:
torontogeo=pd.read_csv(file_path)
torontogeo.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


#### Visualize boroughs on the folium map

In [6]:
# the location info of Toronto
latitude=43.700110
longitude=-79.416300

In [7]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(torontogeo['Latitude'], torontogeo['Longitude'], torontogeo['Borough'], torontogeo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

#### Select boroughs with word 'Toronto' and visualize them 

In [8]:
dft = torontogeo[torontogeo['Borough'].str.contains('Toronto', regex=False, case=False, na=False)]
dft.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
t_borough=dft.reset_index()
t_borough.drop(columns=['index'], axis=1, inplace=True)
print(t_borough.shape)
t_borough.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [ ]:
#save the dataframe
t_borough.to_csv('C://Users/kh116/Desktop/Hyt/IBM Data Science/Course 9-Applied Data Science Capstone/Week 3 Clustering/t_borough.csv')

In [10]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(t_borough['Latitude'], t_borough['Longitude'], t_borough['Borough'], t_borough['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

## 2. Fetch the information of activities around neighborhoods

#### Define Foursquare credentials and versions

In [11]:
CLIENT_ID = '5S50NX55VPFLHVTLIBXW1JN1BWHX20HKZD5E3IBXX4PLFXVU' # your Foursquare ID
CLIENT_SECRET = 'YX2N01VTYZMWPDHH43H3SJJ5CCEKZM32XO2SFBOLO441FND0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5S50NX55VPFLHVTLIBXW1JN1BWHX20HKZD5E3IBXX4PLFXVU
CLIENT_SECRET:YX2N01VTYZMWPDHH43H3SJJ5CCEKZM32XO2SFBOLO441FND0


#### Fetch the information of activities around neighborhoods

In [12]:
import requests

In [13]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
tborough_venues=getNearbyVenues(names=t_borough['Borough'], 
                                latitudes=t_borough['Latitude'], 
                                longitudes=t_borough['Longitude'], 
                                LIMIT=100,
                                radius=500)

Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=5S50NX55VPFLHVTLIBXW1JN1BWHX20HKZD5E3IBXX4PLFXVU&client_secret=YX2N01VTYZMWPDHH43H3SJJ5CCEKZM32XO2SFBOLO441FND0&v=20180605&ll=43.6626956,-79.4000493&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fb0fccffc88>: Failed to establish a new connection: [Errno 110] Connection timed out',))

In [36]:
tborough_venues=pd.read_csv('tborough_venues.csv')

In [37]:
print(tborough_venues.shape)
tborough_venues.head()

(1707, 8)


,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,2,Downtown Toronto,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,4,Downtown Toronto,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [ ]:
#save the dataframe
tborough_venues.to_csv('C://Users/kh116/Desktop/Hyt/IBM Data Science/Course 9-Applied Data Science Capstone/Week 3 Clustering/tborough_venues.csv')

In [17]:
tborough_venues.groupby('Neighborhood').count()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Central Toronto,112,112,112,112,112,112,112
Downtown Toronto,1291,1291,1291,1291,1291,1291,1291
East Toronto,126,126,126,126,126,126,126
West Toronto,178,178,178,178,178,178,178


## 3. Analyze each neighborhood

#### Get dummies of each venues

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(tborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = tborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1707, 237)


,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Calculate the mean of venues of each borough

In [19]:
tborough_grouped=toronto_onehot.groupby('Borough').mean().reset_index()
tborough_grouped

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,...,0.000000,0.008929,0.008929,0.000000,0.008929,0.000000,0.008929,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000775,0.000775,0.000775,0.000775,0.001549,0.002324,0.001549,0.015492,0.001549,...,0.000775,0.000775,0.000775,0.002324,0.013168,0.002324,0.004648,0.006197,0.000775,0.002324
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,...,0.007937,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.011236,0.000000,0.011236,0.005618,0.000000,0.005618


# 4. Clustering boroughs

In [34]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 10

tborough_clustering = toronto_onehot.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tborough_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 1, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

In [38]:
# add clustering labels
tborough_venues.insert(0, 'Cluster Labels', kmeans.labels_)
print(tborough_venues.shape)
tborough_venues.head()

(1707, 9)


,Cluster Labels,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,5,0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,1,1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,0,2,Downtown Toronto,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,0,3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,0,4,Downtown Toronto,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


#### Cluster boroughs

In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
map_clusters

In [40]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [41]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tborough_venues['Neighborhood Latitude'], tborough_venues['Neighborhood Longitude'], tborough_venues['Neighborhood'], tborough_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters